In [55]:
import calibration as _cal 
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from numpy.linalg import matrix_power
from scipy.linalg import expm, expm_frechet
import pygsti
from pygsti.modelmembers.operations import create_from_unitary_mx

In [56]:
def PauliMatrix(i):
    if i == 0:
        return np.array([[1, 0], [0, 1]])
    elif i == 1:
        return np.array([[0, 1], [1, 0]])
    elif i == 2:
        return np.array([[0, -1j], [1j, 0]])
    elif i == 3:
        return np.array([[1, 0], [0, -1]])
    else:
        raise ValueError("i must be 0, 1, 2, or 3.")

def PauliTensor(i, j):
    return np.kron(PauliMatrix(i), PauliMatrix(j))

def PauliMatrix2Q(n):
    i = n // 4
    j = n % 4
    return PauliTensor(i, j)


In [57]:
def make_cz_generator(xstate):
    error_gen = -(1j/2)*sum([xstate[i-1]*PauliMatrix2Q(i) for i in range(16) if i != 0])
    target_gen = -(1j/2)*(- (np.pi/2)*PauliMatrix2Q(0)  
                          + (np.pi/2)*PauliTensor(3, 0) 
                          + (np.pi/2)*PauliTensor(0, 3) 
                          - (np.pi/2)*PauliTensor(3, 3))
    return error_gen + target_gen
    
def make_cz_unitary(xstate):
    return expm(make_cz_generator(xstate))

In [58]:
xstate = np.zeros(15)
cz = make_cz_unitary(xstate)

In [60]:
all_single_qubit_prep_strings = ['0', '1', '+', '-', 'r', 'l']
all_single_qubit_meas_strings = ['0', '1', '+', '-', 'r', 'l']
all_two_qubit_prep_strings = [p1 + p2 for p1 in all_single_qubit_prep_strings for p2 in all_single_qubit_prep_strings]
all_two_qubit_meas_strings = [m1 + m2 for m1 in all_single_qubit_meas_strings for m2 in all_single_qubit_meas_strings]

all_sp_op_strings = ['I.X', 'I.XX', 'I.XXX', 
                     'I.Y', 'I.YY', 'I.YYY',
                     'I.Z', 'I.ZZ', 'I.ZZZ',
                     'X.I', 'XX.I', 'XXX.I',
                     'Y.I', 'YY.I', 'YYY.I',
                     'Z.I', 'ZZ.I', 'ZZZ.I']

U_rx = expm(-1j*np.pi/4*PauliMatrix(1))
U_ry = expm(-1j*np.pi/4*PauliMatrix(2))
U_rz = expm(-1j*np.pi/4*PauliMatrix(3))

def prep_string_to_vec(prep_string):
    prep_comps = []
    for i in range(2):
        if prep_string[i] == '0':
            prep_comps.append(np.array([1, 0]))
        elif prep_string[i] == '1':
            prep_comps.append(np.array([0, 1]))
        elif prep_string[i] == '+':
            prep_comps.append((1/np.sqrt(2))*np.array([1, 1]))
        elif prep_string[i] == '-':
            prep_comps.append((1/np.sqrt(2))*np.array([1, -1]))
        elif prep_string[i] == 'r':
            prep_comps.append((1/np.sqrt(2))*np.array([1, 1j]))
        elif prep_string[i] == 'l':
            prep_comps.append((1/np.sqrt(2))*np.array([1, -1j]))
        else:
            raise ValueError("prep_string must be a valid prep string.")
    return np.kron(prep_comps[0], prep_comps[1])

def meas_string_to_vec(meas_string):
    prep_vec = prep_string_to_vec(meas_string)
    return np.conj(prep_vec).T

def make_single_qubit_sp_op(sp_op_string):
    if sp_op_string == 'I':
        return np.eye(2)
    elif sp_op_string == 'X':
        return U_rx
    elif sp_op_string == 'XX':
        return U_rx@U_rx
    elif sp_op_string == 'XXX':
        return U_rx@U_rx@U_rx
    elif sp_op_string == 'Y':
        return U_ry
    elif sp_op_string == 'YY':
        return U_ry@U_ry
    elif sp_op_string == 'YYY':
        return U_ry@U_ry@U_ry
    elif sp_op_string == 'Z':
        return U_rz
    elif sp_op_string == 'ZZ':
        return U_rz@U_rz
    elif sp_op_string == 'ZZZ':
        return U_rz@U_rz@U_rz
    else:
        raise ValueError("sp_op_string must be a valid sp_op string.")

def sp_op_string_to_matrix(sp_op_string):
    # split into two parts at '.'
    parts = sp_op_string.split('.')
    p0 = make_single_qubit_sp_op(parts[0])
    p1 = make_single_qubit_sp_op(parts[1])
    return np.kron(p0, p1)

def observation_definition(sp_op, prep, meas, depth):
    obs_def = {}
    obs_def['sp_op'] = sp_op_string_to_matrix(sp_op)
    obs_def['prep'] = prep_string_to_vec(prep)
    obs_def['meas'] = meas_string_to_vec(meas)
    obs_def['depth'] = depth
    return obs_def

In [69]:
def make_process_unitary(xstate, sp_op, d):
    u_cz = make_cz_unitary(xstate)
    proc_u = matrix_power(sp_op@u_cz, d)
    return proc_u

def observation_probability(xstate, obs_def):
    sp_op = obs_def['sp_op']
    d = obs_def['depth']
    proc_u = make_process_unitary(xstate, sp_op, d)
    prep = obs_def['prep']
    meas = obs_def['meas']
    return abs(meas@proc_u@prep)**2

def observation_jacobian(xstate, obs_def, epsilon=1e-8):
    sp_op = obs_def['sp_op']
    d = obs_def['depth']
    prep = obs_def['prep']
    meas = obs_def['meas']
    n = len(xstate)
    jacobian = np.zeros((4, n))
    for i in range(n):
        xstate_p = xstate.copy()
        xstate_p[i] += epsilon
        xstate_m = xstate.copy()
        xstate_m[i] -= epsilon
        jacobian[:, i] = (observation_probability(xstate_p, obs_def) - observation_probability(xstate_m, obs_def))/(2*epsilon)
    return jacobian

In [71]:
def observation_fisher_matrix(xstate, obs_def):
    sp_op = obs_def['sp_op']
    d = obs_def['depth']
    prep = obs_def['prep']
    meas = obs_def['meas']
    p = observation_probability(xstate, obs_def)
    jacobian = observation_jacobian(xstate, obs_def)
    if p == 0:
        return np.zeros((15, 15))
    elif p == 1:
        return np.zeros((15, 15))
    else:
        return 1/(p*(1-p))*np.outer(jacobian, jacobian)
    
def observation_mutual_info(xstate, P_covar, obs_def):
    P_covar_inv = np.linalg.inv(P_covar)
    fisher = observation_fisher_matrix(xstate, obs_def)
    return 0.5*np.log(np.linalg.det(P_covar@fisher@P_covar_inv))

In [72]:
xstate = np.zeros(15)
obs_def = observation_definition('I.X', '+0', '00', 1)
print(observation_probability(xstate, obs_def))

0.25


In [73]:
np.linalg.eigvals(observation_fisher_matrix(xstate, obs_def))

array([ 7.99999990e+00, -1.08212392e-16,  6.16297582e-33,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  